# Group 1 - Yelp Review Analysis and Visualization

# 1. Overall Project Objectives  

Yelp is an application to provide the platform for customers to write reviews and provide a star-rating. A research indicates that a one-star increase led to 59% increase in revenue of independent restaurants. Therefore, we see great potential of Yelp dataset as a valuable insights repository.

The main purpose of our project is to conduct thorough analysis on 7 different cuisine types of restaurants which are Korean, Japanese, Chinese, Vietnamese,Thai, French and Italian.

# 2. Description of Data

The Yelp dataset is downloaded from Kaggle website. In total, there are 5,200,000 user reviews, information on 174,000 business. we will focus on two tables which are business table and review table. Attributes of business table are as following:

* business_id: ID of the business 
* name: name of the business
* neighborhood 
* address: address of the business
* city: city of the business
* state: state of the business
* postal_code: postal code of the business
* latitude: latitude of the business
* longitude: longitude of the business
* stars: average rating of the business
* review_count: number of reviews received
* is_open: 1 if the business is open, 0 therwise
* categories: multiple categories of the business


# 3. Direction of Analysis 


**Exploratory data  analysis**
* Count the number of each cuisine type of restaurants
* Count the number of reviews in each cuisine type of restaurants
* Visualize the distribution of restaurants according to the ratings and cuisine types of restaurants.



# 4.  Summary of Progress 

**Selection and Filtering**
* Filter out 50 states of US.  
* Filter out all restaurants of US. 

**Cleaning**
* Categorize all restaurants by cuisine type using the matching keywords.  
* Delete all records with null category. 
* Remove quotation marks of name and address columns.  
* Drop rows with neural label.

		
**Reshaping and Reindexing**
* Reindex the data frame.  
* Build a new column to input the new category name and delete the previous column.  
* Convert array to dataframe. 
		
**Visualization**
* Visualize the distribution of restaurants and reviews by category using seaborn.  
* Visualize the distribution of restaurants and reviews by rating. 
* Visualize top 10 negative words and positive words in each cuisine type.

 

#Data Loading from google drive




In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## Data Preparation

In [ ]:
import pandas as pd
import seaborn as sns
%pylab inline

import requests, re
import pandas as pd
import seaborn as sns
import nltk
import string, itertools
from collections import Counter, defaultdict
from nltk.text import Text
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize, sent_tokenize, regexp_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from gensim.corpora.dictionary import Dictionary
from gensim.models.tfidfmodel import TfidfModel
from sklearn.cluster import KMeans
from wordcloud import WordCloud

Populating the interactive namespace from numpy and matplotlib


### Clean Yelp_business dataset 

In [ ]:
business = pd.read_json('/content/drive/My Drive/yelp_academic_dataset_business.json', lines=True)
business.head()

**Selection and Filtering**

Filter out 50 states of US.

Filter out all restaurants of US.

Delete all records with null category.

Remove quotation marks of name and address columns.



In [ ]:
## filter restaurants of US
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
usa=business.loc[business['state'].isin(states)]
business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...","{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'..."
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,85258,33.569404,-111.890264,5.0,4,1,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Health & Medical, Fitness & Instruction, Yoga,...",None
2,XNoUzKckATkOD1hP6vghZg,Felinus,3554 Rue Notre-Dame O,Montreal,QC,H4C 1P4,45.479984,-73.580070,5.0,5,1,None,"Pets, Pet Services, Pet Groomers",None
3,6OAZjbxqM5ol29BuHsil3w,Nevada House of Hose,1015 Sharp Cir,North Las Vegas,NV,89030,36.219728,-115.127725,2.5,3,0,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Hardware Stores, Home Services, Building Suppl...","{'Monday': '7:0-16:0', 'Tuesday': '7:0-16:0', ..."
4,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,Mesa,AZ,85205,33.428065,-111.726648,4.5,26,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Home Services, Plumbing, Electricians, Handyma...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-16:0', '..."


#Cleaning

Categorize all restaurants by cuisine type using the matching keywords.

Drop rows with neural label.



In [ ]:
### select all restaurants in USA
us_restaurants=usa[usa['categories'].str.contains('Restaurants', na=False)]

## select out 16 cuisine types of restaurants and rename the category
us_restaurants.is_copy=False
us_restaurants['category']=pd.Series()
us_restaurants.loc[us_restaurants.categories.str.contains('American'),'category'] = 'American'
us_restaurants.loc[us_restaurants.categories.str.contains('Mexican'), 'category'] = 'Mexican'
us_restaurants.loc[us_restaurants.categories.str.contains('Italian'), 'category'] = 'Italian'
us_restaurants.loc[us_restaurants.categories.str.contains('Japanese'), 'category'] = 'Japanese'
us_restaurants.loc[us_restaurants.categories.str.contains('Chinese'), 'category'] = 'Chinese'
us_restaurants.loc[us_restaurants.categories.str.contains('Thai'), 'category'] = 'Thai'
us_restaurants.loc[us_restaurants.categories.str.contains('Mediterranean'), 'category'] = 'Mediterranean'
us_restaurants.loc[us_restaurants.categories.str.contains('French'), 'category'] = 'French'
us_restaurants.loc[us_restaurants.categories.str.contains('Vietnamese'), 'category'] = 'Vietnamese'
us_restaurants.loc[us_restaurants.categories.str.contains('Greek'),'category'] = 'Greek'
us_restaurants.loc[us_restaurants.categories.str.contains('Indian'),'category'] = 'Indian'
us_restaurants.loc[us_restaurants.categories.str.contains('Korean'),'category'] = 'Korean'
us_restaurants.loc[us_restaurants.categories.str.contains('Hawaiian'),'category'] = 'Hawaiian'
us_restaurants.loc[us_restaurants.categories.str.contains('African'),'category'] = 'African'
us_restaurants.loc[us_restaurants.categories.str.contains('Spanish'),'category'] = 'Spanish'
us_restaurants.loc[us_restaurants.categories.str.contains('Middle_eastern'),'category'] = 'Middle_eastern'
us_restaurants.category[:20]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


8             NaN
20            NaN
33            NaN
36        Mexican
41     Vietnamese
42       American
44        Mexican
46       American
49            NaN
50        Chinese
54            NaN
59            NaN
61       American
63            NaN
83       American
86            NaN
89           Thai
100           NaN
101           NaN
102           NaN
Name: category, dtype: object

In [ ]:
## drop null values in category, delete original column categories and reset the index
us_restaurants=us_restaurants.dropna(axis=0, subset=['category'])
del us_restaurants['categories']
us_restaurants=us_restaurants.reset_index(drop=True)
us_restaurants.head(10)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,hours,category
0,fnZrZlqW1Z8iWgTVDfv_MA,Carl's Jr,9595 W Tropicana Ave,Las Vegas,NV,89147,36.099738,-115.301568,2.5,15,0,"{'Alcohol': 'u'none'', 'WiFi': 'u'no'', 'GoodF...",None,Mexican
1,98hyK2QEUeI8v2y0AghfZA,Pho Lee's Vietnamese Restaurant,"1541 E 38th St, Ste 101",Cleveland,OH,44114,41.512155,-81.663332,4.5,23,1,"{'BusinessParking': 'None', 'Ambience': '{'tou...","{'Monday': '11:0-20:0', 'Wednesday': '11:0-20:...",Vietnamese
2,fhNf_sg-XzZ3e7HEVGuOZg,Meat Chix And Wieners,6530 S Decatur Blvd,Las Vegas,NV,89118,36.071196,-115.207463,3.0,10,0,"{'RestaurantsPriceRange2': '2', 'BusinessParki...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",American
3,Ga2Bt7xfqoggTypWD5VpoQ,Amando's Bros,2602 W Southern Ave,Tempe,AZ,85282,33.393199,-111.976270,4.0,9,0,"{'Caters': 'False', 'RestaurantsGoodForGroups'...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",Mexican
4,xFc50drSPxXkcLvX5ygqrg,Boomer's Sweet Home Chicago,"5932 W Bell Rd, Ste D-109",Glendale,AZ,85308,33.640391,-112.187028,3.5,12,0,"{'BusinessAcceptsCreditCards': 'True', 'Busine...",None,American
5,Sd75ucXKoZUM2BEfBHFUOg,China Gourmet,"3460 E Southern Ave, Ste 109",Mesa,AZ,85204,33.394863,-111.756036,3.0,13,0,"{'RestaurantsPriceRange2': '1', 'RestaurantsAt...",None,Chinese
6,-qjn24n8HYF6It9GQrQntw,Five Guys Burgers and Fries,2130 E Arbors Dr,Charlotte,NC,28262,35.338070,-80.757397,4.0,15,0,"{'RestaurantsReservations': 'False', 'Restaura...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",American
7,7j0kor_fkeYhyEpXh4OpnQ,Scramblers - Strongsville,16726 Pearl Rd,Strongsville,OH,44136,41.295689,-81.835722,3.5,81,1,"{'BusinessAcceptsCreditCards': 'True', 'Ambien...","{'Monday': '6:30-15:0', 'Tuesday': '6:30-15:0'...",American
8,j9bWpCRwpDVfwVT_V85qeA,Papaya Thai,2706 E University Dr,Mesa,AZ,85213,33.423705,-111.772890,2.5,130,1,"{'Caters': 'False', 'OutdoorSeating': 'False',...","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",Thai
9,8nP8ghEpT6WFcM6tfqAaGA,Sugar Bowl,4005 N Scottsdale Rd,Scottsdale,AZ,85251,33.493775,-111.925933,3.5,720,1,"{'RestaurantsAttire': ''casual'', 'Alcohol': '...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",American


In [ ]:
## check total number of us restaurants
us_restaurants.shape

(22779, 14)

In [ ]:
## check whether has duplicated business id
us_restaurants.business_id.duplicated().sum()

0

In [ ]:
## check the datatype
us_restaurants.dtypes

business_id      object
name             object
address          object
city             object
state            object
postal_code      object
latitude        float64
longitude       float64
stars           float64
review_count      int64
is_open           int64
attributes       object
hours            object
category         object
dtype: object

In [ ]:
## check missing values
us_restaurants.isnull().sum()

business_id        0
name               0
address            0
city               0
state              0
postal_code        0
latitude           0
longitude          0
stars              0
review_count       0
is_open            0
attributes       223
hours           3773
category           0
dtype: int64

** Yelp filter and visualization **

In [ ]:
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#to make the interactive maps
import folium
from folium.plugins import FastMarkerCluster

from ipywidgets import widgets, interact
from IPython.display import display

In [ ]:
#us_restaurants.drop('attributes',axis=1, inplace=True)

In [ ]:
min_rating = 0 
category = 'American'



**`Visualizing entire restaurant dataset `**

In [ ]:
if(us_restaurants.empty == False):
 
  latitude = us_restaurants['latitude'].tolist()
  longi = us_restaurants['longitude'].tolist()
  locations = list(zip(latitude, longi))

  map1 = folium.Map(location=[35, -100], zoom_start=4)
  FastMarkerCluster(data=locations).add_to(map1)
  display(map1)
else:
   print('No Results Found')

In [ ]:
def f1(x):
  global min_rating
  min_rating = x

def f(x):
  global category
  category = x

print("Select category")
interact(f, x=['category','American','Mexican','Italian','Japanese','Chinese','Thai','Mediterranean','Indian','French','Vietnamese','Korean']);

print("Select restaurants with particular minimum rating")
interact(f1,  x=widgets.IntSlider(min=0, max=5, step=1, value=0))

Select category


interactive(children=(Dropdown(description='x', options=('category', 'American', 'Mexican', 'Italian', 'Japane…

Select restaurants with particular minimum rating


interactive(children=(IntSlider(value=0, description='x', max=5), Output()), _dom_classes=('widget-interact',)…

<function __main__.f1>

In [ ]:
rating_df = us_restaurants.loc[us_restaurants['stars'] >= min_rating]
category_df = us_restaurants.loc[us_restaurants['category'] == category]
merged_df= pd.merge(rating_df, category_df, on = 'business_id')


In [ ]:
merged_df.dtypes


business_id        object
name_x             object
address_x          object
city_x             object
state_x            object
postal_code_x      object
latitude_x        float64
longitude_x       float64
stars_x           float64
review_count_x      int64
is_open_x           int64
attributes_x       object
hours_x            object
category_x         object
name_y             object
address_y          object
city_y             object
state_y            object
postal_code_y      object
latitude_y        float64
longitude_y       float64
stars_y           float64
review_count_y      int64
is_open_y           int64
attributes_y       object
hours_y            object
category_y         object
dtype: object

In [ ]:
if(merged_df.empty == False):
 
  latitude = merged_df['latitude_x'].tolist()
  longi = merged_df['longitude_x'].tolist()
  locations = list(zip(latitude, longi))

  map2 = folium.Map(location=[35, -100], zoom_start=4)
  FastMarkerCluster(data=locations).add_to(map2)
  display(map2)
else:
   print('No Results Found')

No Results Found


Heat Map

In [ ]:
from folium.plugins import HeatMap

In [ ]:

map3 = folium.Map(location=[35, -100], zoom_start=4)
HeatMap(locations).add_to(map3)
display(map3)

## Neighbourhood query using KD tree

In [ ]:
from scipy import spatial

In [ ]:
text = widgets.Text()
display(text)

def handle_submit(sender): 
  global search_lat_lon
  search_lat_lon = text.value
text.on_submit(handle_submit)

Text(value='')

In [ ]:
!pip install utm
import utm
input_lat, input_long, radius = list(map(float,search_lat_lon.split(',')))
search_points = [input_lat, input_long]
x1, y1, zone, ut = utm.from_latlon(input_lat, input_long)
search_lat_lon = [x1, y1]

In [ ]:

lats = us_restaurants['latitude'].tolist()
lons = us_restaurants['longitude'].tolist()
locations = list(zip(lats, lons))

x= []
y= []

for tup in locations:
  tup_temp = utm.from_latlon(tup[0], tup[1])
  x.append(tup_temp[0])
  y.append(tup_temp[1])

us_restaurants['x'] = x
us_restaurants['y'] = y

kdtree_points = list(zip(x,y))

In [ ]:
from scipy import spatial
import numpy as np

tree = spatial.KDTree(kdtree_points)
indices = tree.query_ball_point(search_lat_lon, radius)

radial_search_results = us_restaurants.iloc[indices]



In [ ]:
#indices
#search_lat_lon
radial_search_results.head(5)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,hours,category,x,y
16425,gcNC4k7TZJVX_1YHdIkDNQ,Sushi Catcher,6334 W Bell Rd,Glendale,AZ,85308,33.639282,-112.195893,4.0,437,1,"{'RestaurantsGoodForGroups': 'True', 'HasTV': ...","{'Monday': '0:0-0:0', 'Tuesday': '16:30-21:0',...",Japanese,389094.118260,3.722803e+06
4,xFc50drSPxXkcLvX5ygqrg,Boomer's Sweet Home Chicago,"5932 W Bell Rd, Ste D-109",Glendale,AZ,85308,33.640391,-112.187028,3.5,12,0,"{'BusinessAcceptsCreditCards': 'True', 'Busine...",None,American,389917.669820,3.722916e+06
1160,thlAnPN1ApoNxSnok_fcvA,Black Bear Diner,6039 W Bell Rd,Glendale,AZ,85308,33.638271,-112.190578,4.0,379,1,"{'Caters': 'False', 'Alcohol': ''beer_and_wine...","{'Monday': '6:0-20:0', 'Tuesday': '6:0-22:0', ...",American,389585.801432,3.722685e+06
3761,IMu-TDHot24Y0X7ESLxHyA,Macayo's Mexican Food,6012 W Bell Rd,Glendale,AZ,85308,33.639221,-112.188989,3.0,271,1,"{'RestaurantsAttire': ''casual'', 'BusinessAcc...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",Mexican,389734.345836,3.722789e+06
8262,A2J7d9UxD40rVgjaKflVhw,Al's Gyro's,"5932 W Bell Rd, Ste D109",Glendale,AZ,85308,33.640391,-112.187028,4.5,120,1,"{'RestaurantsTakeOut': 'True', 'WiFi': 'u'free...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-20:0', ...",Greek,389917.669820,3.722916e+06


In [ ]:
locations_kd = list(zip(radial_search_results['latitude'].tolist(), radial_search_results['longitude'].tolist()))

In [ ]:
search_points 

[33.640391, -112.187028]

In [ ]:
map6 = folium.Map(location=[radial_search_results['latitude'].iloc[0], radial_search_results['longitude'].iloc[0]], zoom_start=16)
counter = 0
for lat, lon in locations_kd:
        folium.Marker([lat, lon],popup= "name: {0}, category: {1} \n location: {2},{3} " \
                      .format(radial_search_results['name'].iloc[counter],\
                              radial_search_results['category'].iloc[counter],\
                              lat, lon\
                              )).add_to(map6)
        counter += 1

map6

## Exploratory Data Analysis

### Restaurants Distribution

#### Distribution of restaurants in each category

In [ ]:
plt.style.use('ggplot')

In [ ]:
plt.figure(figsize=(11,7))
grouped = us_restaurants.category.value_counts()
sns.countplot(y='category',data=us_restaurants, 
              order = grouped.index, palette= sns.color_palette("RdBu_r", len(grouped)))
plt.xlabel('Number of restaurants', fontsize=14, labelpad=10)
plt.ylabel('Category', fontsize=14)
plt.title('Count of Restaurants by Category', fontsize=15)
plt.tick_params(labelsize=14)
for  i, v in enumerate(us_restaurants.category.value_counts()):
    plt.text(v, i+0.15, str(v), fontweight='bold', fontsize=14)

Categories in dark blue color have the largest number of restaurants. On the contrary, categories in dark red color have the least number of restaurants. The top 5 type of restaurants are American, Mexican, Italian, Chinese and Japanese. 

#### Top 10 cities with most restaurants

In [ ]:
plt.figure(figsize=(11,6))
grouped = us_restaurants.city.value_counts()[:10]
sns.barplot(grouped.index, grouped.values, palette=sns.color_palette("GnBu_r", len(grouped)))
plt.ylabel('Number of restaurants', fontsize=14, labelpad=10)
plt.xlabel('City', fontsize=14, labelpad=10)
plt.title('Count of Restaurants by City (Top 10)', fontsize=15)
plt.tick_params(labelsize=14)
plt.xticks(rotation=15)
for  i, v in enumerate(grouped):
    plt.text(i, v*1.02, str(v), horizontalalignment ='center',fontweight='bold', fontsize=14)

#### Distribution of restaurants in each state

In [ ]:
plt.figure(figsize=(11,6))
grouped = us_restaurants.state.value_counts()
sns.barplot(grouped.index, grouped.values,palette=sns.color_palette("GnBu_r", len(grouped)) )
plt.ylabel('Number of restaurants', fontsize=14)
plt.xlabel('State', fontsize=14)
plt.title('Count of Restaurants by State', fontsize=15)
plt.tick_params(labelsize=14)
for  i, v in enumerate(grouped):
    plt.text(i, v*1.02, str(v), horizontalalignment ='center', fontweight='bold', fontsize=14)

### Reviews Distribution

#### Distribution of reviews by cuisine type

In [ ]:
plt.figure(figsize=(11,7))
grouped = us_restaurants.groupby('category')['review_count'].sum().sort_values(ascending = False)
sns.barplot(y=grouped.index, x= grouped.values, palette= sns.color_palette("RdBu_r", len(grouped)) )
plt.ylabel('Category', fontsize=14)
plt.xlabel('Count of reviews', fontsize=14)
plt.title('Count of Reviews by Cuisine Type', fontsize=15)
for i,v in enumerate(grouped):
    plt.text(v, i+0.15, str(v),fontweight='bold', fontsize=14)
plt.tick_params(labelsize=14)

#### Top 10 cities with most reviews

In [ ]:
plt.figure(figsize=(11,6))
grouped = us_restaurants.groupby('city')['review_count'].sum().sort_values(ascending=False)[:10]
sns.barplot(grouped.index, grouped.values, palette=sns.color_palette("GnBu_r", len(grouped)) )
plt.xlabel('City', labelpad=10, fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.title('Count of Reviews by City (Top 10)', fontsize=15)
plt.tick_params(labelsize=14)
plt.xticks(rotation=15)
for  i, v in enumerate(grouped):
    plt.text(i, v*1.02, str(v), horizontalalignment ='center',fontweight='bold', fontsize=14)

#### Top 9 restaurants with most reviews

In [ ]:
plt.figure(figsize=(11,6))
grouped = us_restaurants[['name','review_count']].sort_values(by='review_count', ascending=False)[:9]
sns.barplot(x=grouped.review_count, y = grouped.name, palette=sns.color_palette("GnBu_r", len(grouped)), ci=None)
plt.xlabel('Count of Review', labelpad=10, fontsize=14)
plt.ylabel('Restaurants', fontsize=14)
plt.title('TOP 9 Restaurants with Most Reviews', fontsize=15)
plt.tick_params(labelsize=14)
plt.xticks(rotation=15)
for  i, v in enumerate(grouped.review_count):
    plt.text(v, i, str(v), fontweight='bold', fontsize=14)

### Ratings Distribution

#### Distribution of ratings by restaurants

In [ ]:
plt.figure(figsize=(11,6))
grouped = us_restaurants.stars.value_counts().sort_index()
sns.barplot(grouped.index, grouped.values, palette=sns.color_palette("RdBu_r", len(grouped)))
plt.xlabel('Average Rating', labelpad=10, fontsize=14)
plt.ylabel('Count of restaurants', fontsize=14)
plt.title('Count of Restaurants against Ratings', fontsize=15)
plt.tick_params(labelsize=14)
for  i, v in enumerate(grouped):
    plt.text(i, v*1.02, str(v), horizontalalignment ='center',fontweight='bold', fontsize=14)